# Thermal Modeling with Pulsim

This notebook demonstrates thermal-electrical co-simulation for power electronics.

## Contents
1. Thermal Network Basics
2. MOSFET with Thermal Model
3. Temperature-Dependent Parameters
4. Thermal Transient Analysis

## 1. Thermal Network Basics

Pulsim uses Foster RC networks to model thermal behavior:

```
         R_th1        R_th2        R_th3
P_loss ──/\/\/──┬──/\/\/──┬──/\/\/──┬── T_ambient
                │         │         │
               ═══       ═══       ═══
               C1        C2        C3
                │         │         │
               GND       GND       GND
```

The transient thermal impedance is:
$$Z_{th}(t) = \sum_{i=1}^{n} R_{th,i} \cdot (1 - e^{-t/\tau_i})$$

where $\tau_i = R_{th,i} \cdot C_{th,i}$

In [ ]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt

# Typical Foster parameters for a TO-220 package MOSFET
# From IRF540N datasheet (approximate)
R_th = [0.18, 0.42, 0.55, 0.35]  # K/W
tau = [0.5e-3, 5e-3, 50e-3, 500e-3]  # seconds

R_th_total = sum(R_th)
print(f"Total thermal resistance (junction-to-case): {R_th_total:.2f} K/W")

# Plot thermal impedance
t = np.logspace(-4, 1, 1000)  # 0.1ms to 10s
Zth = np.zeros_like(t)
for r, tau_i in zip(R_th, tau):
    Zth += r * (1 - np.exp(-t / tau_i))

plt.figure(figsize=(10, 6))
plt.loglog(t, Zth, 'b-', linewidth=2)
plt.axhline(y=R_th_total, color='r', linestyle='--', label=f'Rth_jc = {R_th_total:.2f} K/W')
plt.xlabel('Time (s)')
plt.ylabel('Thermal Impedance (K/W)')
plt.title('Transient Thermal Impedance (Junction-to-Case)')
plt.legend()
plt.grid(True, which='both', alpha=0.5)
plt.xlim([1e-4, 10])
plt.show()

## 2. MOSFET with Thermal Model

Let's simulate a MOSFET conducting DC current with thermal coupling.

In [ ]:
# Create circuit with thermal-electrical coupling
circuit = pulsim.Circuit("MOSFET Thermal Test")

# Power supply
circuit.add_voltage_source("Vds", "drain", "0", 50.0)  # 50V drain
circuit.add_voltage_source("Vgs", "gate", "0", 10.0)   # 10V gate (fully on)

# MOSFET with thermal model
circuit.add_mosfet("M1", "drain", "gate", "source", "0",
    model="IRF540N",
    rds_on=0.044,  # 44mΩ at 25°C
    thermal={
        "node": "Tj",
        "rth": R_th,
        "tau": tau
    },
    tc1=0.007  # Temperature coefficient: Rds_on increases ~0.7%/°C
)

# Current sensing resistor (small)
circuit.add_resistor("R_sense", "source", "0", 0.01)  # 10mΩ

# Heatsink and ambient
circuit.add_thermal_resistor("Rth_cs", "Tj", "Tc", 0.5)    # Case-to-sink: 0.5 K/W
circuit.add_thermal_resistor("Rth_sa", "Tc", "Ta", 2.0)    # Sink-to-ambient: 2 K/W
circuit.add_thermal_source("T_ambient", "Ta", "0", 25.0)   # 25°C ambient

print("Circuit with thermal model created!")

In [ ]:
# Simulate for 10 seconds to reach thermal steady state
options = pulsim.SimulationOptions(
    stop_time=10.0,
    timestep=1e-3,  # 1ms timestep (thermal time constants are much slower)
    abstol=1e-12,
    reltol=1e-3
)

print("Running thermal simulation...")
result = pulsim.simulate(circuit, options)
print("Done!")

In [ ]:
# Extract results
time = result.time
Tj = result.temperatures["Tj"]  # Junction temperature
Tc = result.temperatures["Tc"]  # Case temperature
Id = result.currents["R_sense"] / 0.01  # Drain current

# Calculate power dissipation (approximate)
Vds_on = result.voltages["drain"] - result.voltages["source"]
P_loss = Vds_on * Id

# Plot
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

# Temperature
axes[0].plot(time, Tj, 'r-', label='Junction (Tj)', linewidth=2)
axes[0].plot(time, Tc, 'b-', label='Case (Tc)', linewidth=2)
axes[0].axhline(y=25, color='g', linestyle='--', label='Ambient')
axes[0].set_ylabel('Temperature (°C)')
axes[0].set_title('MOSFET Temperature Rise')
axes[0].legend()
axes[0].grid(True)

# Current
axes[1].plot(time, Id, 'g-', linewidth=2)
axes[1].set_ylabel('Drain Current (A)')
axes[1].set_title('Drain Current (decreases as Rds_on increases with temperature)')
axes[1].grid(True)

# Power
axes[2].plot(time, P_loss, 'm-', linewidth=2)
axes[2].set_ylabel('Power (W)')
axes[2].set_xlabel('Time (s)')
axes[2].set_title('Power Dissipation')
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate steady-state values
Tj_ss = Tj[-1]
Tc_ss = Tc[-1]
P_ss = P_loss[-1]
Id_ss = Id[-1]

# Total thermal resistance
Rth_total = R_th_total + 0.5 + 2.0  # Junction-case + case-sink + sink-ambient

print("Steady-State Analysis:")
print(f"  Junction temperature: {Tj_ss:.1f}°C")
print(f"  Case temperature: {Tc_ss:.1f}°C")
print(f"  Temperature rise: {Tj_ss - 25:.1f}°C")
print(f"  Power dissipation: {P_ss:.2f}W")
print(f"  Drain current: {Id_ss:.3f}A")
print(f"\nTheoretical check:")
print(f"  Expected ΔT = P × Rth_total = {P_ss:.2f} × {Rth_total:.2f} = {P_ss * Rth_total:.1f}°C")
print(f"  Simulated ΔT = {Tj_ss - 25:.1f}°C")

## 3. Temperature-Dependent Parameters

The MOSFET Rds_on increases with temperature:
$$R_{ds(on)}(T) = R_{ds(on)}(25°C) \times [1 + TC_1 \times (T - 25)]$$

In [ ]:
# Calculate Rds_on change
Rds_on_25C = 0.044  # Ω at 25°C
TC1 = 0.007  # 0.7%/°C

Rds_on_hot = Rds_on_25C * (1 + TC1 * (Tj_ss - 25))

print(f"Rds_on at 25°C: {Rds_on_25C*1000:.1f} mΩ")
print(f"Rds_on at {Tj_ss:.0f}°C: {Rds_on_hot*1000:.1f} mΩ")
print(f"Increase: {(Rds_on_hot/Rds_on_25C - 1)*100:.1f}%")

In [ ]:
# Plot Rds_on vs temperature characteristic
T_range = np.linspace(25, 175, 100)
Rds_vs_T = Rds_on_25C * (1 + TC1 * (T_range - 25))

plt.figure(figsize=(10, 6))
plt.plot(T_range, Rds_vs_T * 1000, 'b-', linewidth=2)
plt.axvline(x=Tj_ss, color='r', linestyle='--', label=f'Operating point: {Tj_ss:.0f}°C')
plt.axhline(y=Rds_on_hot*1000, color='r', linestyle='--')
plt.scatter([Tj_ss], [Rds_on_hot*1000], color='r', s=100, zorder=5)

plt.xlabel('Junction Temperature (°C)')
plt.ylabel('Rds_on (mΩ)')
plt.title('MOSFET Rds_on vs Temperature')
plt.legend()
plt.grid(True)
plt.xlim([25, 175])
plt.show()

## 4. Thermal Transient Analysis

Let's analyze the thermal response to pulsed power.

In [ ]:
# Create a pulsed power test circuit
circuit2 = pulsim.Circuit("Pulsed Power Test")

# Pulsed voltage source (simulates switching)
circuit2.add_voltage_source("Vds", "drain", "0", 50.0,
    waveform={"type": "pulse", "v1": 0, "v2": 50, 
              "period": 1.0, "pw": 0.1})  # 10% duty cycle, 1Hz
circuit2.add_voltage_source("Vgs", "gate", "0", 10.0)

# MOSFET with thermal
circuit2.add_mosfet("M1", "drain", "gate", "source", "0",
    model="IRF540N",
    rds_on=0.044,
    thermal={"node": "Tj", "rth": R_th, "tau": tau},
    tc1=0.007
)

circuit2.add_resistor("R_load", "source", "0", 5.0)  # Load
circuit2.add_thermal_resistor("Rth_cs", "Tj", "Tc", 0.5)
circuit2.add_thermal_resistor("Rth_sa", "Tc", "Ta", 2.0)
circuit2.add_thermal_source("T_ambient", "Ta", "0", 25.0)

# Simulate
options2 = pulsim.SimulationOptions(
    stop_time=30.0,  # 30 seconds
    timestep=1e-3
)

print("Running pulsed power simulation...")
result2 = pulsim.simulate(circuit2, options2)
print("Done!")

In [ ]:
# Plot pulsed response
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Temperature
axes[0].plot(result2.time, result2.temperatures["Tj"], 'r-', label='Tj', linewidth=1)
axes[0].plot(result2.time, result2.temperatures["Tc"], 'b-', label='Tc', linewidth=1)
axes[0].set_ylabel('Temperature (°C)')
axes[0].set_title('Temperature Response to Pulsed Power (10% duty cycle)')
axes[0].legend()
axes[0].grid(True)

# Power
Vds2 = result2.voltages["drain"] - result2.voltages["source"]
Id2 = (50 - Vds2) / 5.0  # Approximate
P2 = Vds2 * Id2

axes[1].plot(result2.time, P2, 'm-', linewidth=1)
axes[1].set_ylabel('Power (W)')
axes[1].set_xlabel('Time (s)')
axes[1].set_title('Instantaneous Power Dissipation')
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate peak and average temperatures
# Use last few cycles for steady-state
ss_mask = result2.time > 20
Tj_ss2 = result2.temperatures["Tj"][ss_mask]

Tj_peak = np.max(Tj_ss2)
Tj_min = np.min(Tj_ss2)
Tj_avg = np.mean(Tj_ss2)
Tj_ripple = Tj_peak - Tj_min

print("Pulsed Operation (Steady State):")
print(f"  Peak junction temperature: {Tj_peak:.1f}°C")
print(f"  Minimum junction temperature: {Tj_min:.1f}°C")
print(f"  Average junction temperature: {Tj_avg:.1f}°C")
print(f"  Temperature ripple: {Tj_ripple:.1f}°C")

## Summary

In this notebook, we covered:
- Foster thermal network modeling
- MOSFET thermal-electrical co-simulation
- Temperature-dependent Rds_on
- Transient thermal analysis with pulsed power

Key takeaways:
- Thermal models are essential for accurate power loss prediction
- Rds_on increases significantly with temperature (positive temperature coefficient)
- Foster networks capture transient thermal behavior with multiple time constants
- Junction temperature ripple depends on switching frequency and thermal impedance

**Next:** [Parameter Sweeps Tutorial](04_parameter_sweeps.ipynb)